# Loading saved models to neuefische's MLFlow AWS Server 

In [1]:
import os
import pickle
import tensorflow as tf
import mlflow
from config import EXPERIMENT_NAME
TRACKING_URI = open("../.mlflow_uri").read().strip()

### Set paths

In [ ]:
# Get current absolute path of parent folder of this file
path_notebooks = os.path.dirname(os.path.abspath('add_models_to_mlflow'))

# Get path of parent directory, i.e. path to repo
path = os.path.abspath(os.path.join(path_notebooks, os.pardir))

### List saved models

In [ ]:
# list available models stored in 'saved_model' (without hidden folders)
x = os.listdir(path + '/saved_model')
x = [i for i in x if not i.startswith('.')]
print(*x, sep = "\n")

## Step 1) Start MLFlow Run

In [2]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.start_run(run_name='base model test-extern')

run = mlflow.active_run()

## Step 2) Load model & setup

In [3]:
# Load saved model
model_name = 'baseline_kaggle_p100_e2'
model = tf.keras.models.load_model(path + '/saved_model/' + model_name)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-11-07 18:12:15.512876: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-07 18:12:15.512957: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# load training history and show keys
with open(path + '/saved_model/' + model_name + '/trainHistoryDict', "rb") as file_pi:
    history = pickle.load(file_pi)

history.keys()

In [6]:
# load training parameters and show keys
with open(path + '/saved_model/' + model_name + '/trainParamsDict', "rb") as file_pi:
    model_params = pickle.load(file_pi)

model_params.keys()

## Step 3) Set MLFlow logging parameters

In [8]:
# setting parameters that should be logged on MLFlow
params = {
      #"dataset": model_params['ds'],
      "batch_size": model_params['bs'],
      "image_size": model_params['img_height'],
      "epochs": model_params['eps'],
      "learning_rate": model_params['lr'],
  }

In [10]:
#logging params to mlflow
mlflow.log_params(params)

#setting tags
mlflow.set_tag("running_from_jupyter", "True")

#logging metrics - last epoch only
mlflow.log_metric("train-" + "accuracy", history['accuracy'][-1])
mlflow.log_metric("valid-" + "accuracy", history['val_accuracy'][-1])
mlflow.log_metric("train-" + "loss", history['loss'][-1])
mlflow.log_metric("valid-" + "loss", history['val_loss'][-1])

# logging the model to mlflow will not work without a AWS Connection setup.. too complex for now
# but possible if running mlflow locally

# log model (only working locally, but not with server)
#mlflow.sklearn.log_model(reg, "model")
#mlflow.keras.log_model(model ,'InceptionV3')

## Step 4) Stop MLFlow Run and get statistics

In [ ]:
# stop run
mlflow.end_run()

In [11]:
# Print stats
mlflow.get_run(run_id=run.info.run_id)

<Run: data=<RunData: metrics={'train-accuracy': 0.3838529884815216,
 'train-loss': 69.74864196777344,
 'valid-accuracy': 0.4154411852359772,
 'valid-loss': 81.21258544921875}, params={'batch_size': '32',
 'epochs': '2',
 'image_size': '100',
 'learning_rate': '0.03'}, tags={'mlflow.runName': 'base model test-extern',
 'mlflow.source.git.commit': 'd31869fcff94d5ed009089d02d3c76d1ab5a4632',
 'mlflow.source.name': '/Users/philipp/neuefische/ds-capstone-for-the-birds/.venv/lib/python3.9/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'philipp',
 'running_from_jupyter': 'True'}>, info=<RunInfo: artifact_uri='s3://neuefische-mlflow/mlflow-artifacts/59/f873d55fc7ca49e08049ee5475e4d16b/artifacts', end_time=1667841276928, experiment_id='59', lifecycle_stage='active', run_id='f873d55fc7ca49e08049ee5475e4d16b', run_uuid='f873d55fc7ca49e08049ee5475e4d16b', start_time=1667841132391, status='FINISHED', user_id='philipp'>>